# Linear Model 

## Background
A single-input, single-output linear function is simple. 
Its two parameters, weight(slope) and bias(y-intercept), tell the entire story of the relationship between the input and the output. 
It is yet powerful.
It reveals natural philosophy with mathematical principles. 
- Galileo Galilei defined the relationship of an object's velocity and its constant acceleration.
- Johannes Kepler described a general planet's motion given the square of its orbital period and the cube of the length of the semi-major axis of its orbit. 
- Isaac Newton connected the acceleration of an object to the net force acting on it and its mass.

It is also the simplest format of an artificial neural network model.
It serves as the basic building block for more complex deep learning models.
By searching for an appropriate combination of the weight and bias parameters, an intimate relationship between two quantities will be established.
Now, let's practice training a deep learning model.
In the following exercises, a linear model will be trained to predict a used car's price based on its age.

## Objectives
- Be able to train a neural network model with the simplest form: single-in, single-out linear function.
    - Initialize weight and bias parameters.
    - Use a loss function to evaluate the model's performance.
    - Optimize the weight and bias parameters using gradient descent algorithm.
- Get more used to vectorization using NumPy.

## Instructions
- Write your code only between the commented lines: `###START CODE HERE###` and `###END CODE HERE###`.
- Replace `None`s with appropriate variables or operations.
- <font color='red'>Modify code out of the designated area at your own risk.</font>
- Reference answers are provided after a certain coding blocks.

## Exercises:
1. <font color='violet'>(10%) Pre-Process Data</font>
2. <font color='violet'>(10%) Create a Linear Model</font>
3. <font color='violet'>(10%) Define Mean Square Error</font>
4. <font color='violet'>(20%) Compute Loss Derivatives</font>
5. <font color='violet'>(40%) Implement Gradient Descent Algorithm</font>
6. <font color='violet'>(10%) Test Model with New Data</font>





## 1 - Load Data
Load the dataset: $\mathcal{D} = \{\mathbf{x}, \mathbf{y}\} = \{(^{(1)} x, ^{(1)} y), (^{(2)} x, ^{(2)} y), ..., (^{(M)} x, ^{(M)} y)\}$, which contains $M$ samples (cars).
Each sample consists with a feature (age): $^{(m)} x$, and the corresponding label (price): $^{(m)} y$, where $m \in \{1, 2, ..., M\}$.

The raw data is stored in numpy arrays. 
Let's first load these arrays. 
Then, visulize the raw data.


In [ ]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt

# Load the  raw data (year, price)
X_raw = np.load("ages_train.npy")
y_raw = np.load("prices_train.npy")
print(f"dimension of raw features: {X_raw.ndim}, shape of raw features: {X_raw.shape}")
print(f"dimension of raw labels: {y_raw.ndim}, shape of raw labels: {y_raw.shape}")
print(f"fisrt 5 samples: {X_raw[:5], y_raw[:5]}")

# Visualize the  raw data
plt.xlabel("Age (years)")
plt.ylabel("Price ($)")
plt.plot(X_raw, y_raw, '.', markersize=1)

As the raw data (feature and label) have different scales, we would like to rescale all the features (ages) and the labels (prices) to roughly between 0 and 10. 

In [ ]:
X_rescale = X_raw / 10    # rescale to per decade
y_rescale = y_raw / 1e4  # rescale to per $10,000
print(f"fisrt 5 rescaled samples: {X_rescale[:5], y_rescale[:5]}")


### <font color='violet'>(10%) Exercise 1: Pre-Process Data</font>
Let's reshape the raw feature and label arrays into 2-dimensional (2d) numpy arrays (So that they look the same as the column vectors in your linear algebra class). After reshaping, the 1st dimension will contain $M$ elements, and the 2nd dimension will contain only 1 element.  


In [ ]:
### START CODE HERE ### (≈ 2 line of code)
X_reshape = None
y_reshape = None
### END CODE HERE ###
print(f"dimension of processed features: {X_reshape.ndim}, shape of processed features: {X_reshape.shape}")
print(f"dimension of processed labels: {y_reshape.ndim}, shape of processed labels: {y_reshape.shape}")
print(f"fisrt 5 processed samples: \n{X_reshape[:5]} \n\n{y_reshape[:5]}")

# Visualize the processed data
plt.xlabel("Age (decades)")
plt.ylabel("Price ($10,000)")
plt.plot(X_reshape, y_reshape, '.', markersize=1)


**Expected Output**:
>
```console
dimension of processed features: 2, shape of processed features: (268577, 1)
dimension of processed labels: 2, shape of processed labels: (268577, 1)
fisrt 5 processed samples: 
[[1.3]
 [0.6]
 [0.5]
 [0.4]
 [0.9]] 

[[0.35  ]
 [2.6   ]
 [2.4971]
 [3.999 ]
 [0.8495]]
```

## 2 - Create a Linear Model
A linear model provides a simple yet powerful method for predicting an outcome $\hat{\mathbf{y}}$ given an input feature $\mathbf{x}$.
The relationship between the input and the output is governed by the weight parameter, $w$ and bias parameter, $b$. 
 
$$\hat{\mathbf{y}} = w \mathbf{x} + b$$

### <font color='violet'>(10%) Exercise 2: Create a Linear Model</font>
Define a python function for the linear model.
- In this assignment, the age of a car is the input feature of a linear model.
- The goal is to predict the car's selling price.
- The `linear` function is supposed to deal with arrays.  

In [ ]:
def linear(inputs, weight, bias):
    """
    Linear model function
        Args:
            inputs: input feature, 2d-array
            weight: slope, scalar
            bias: y-interception, scalar
        Returns:
            outputs: predicted output, 2d-array
    """
    ### START CODE HERE ### (≈ 1 line of code)
    outputs = None
    ### END CODE HERE ###
    return outputs

# Sanity check
print(f"The model's output from 4 input values: \n{linear(np.linspace(-0.2, 0.2, 4).reshape(-1, 1), 2, -3)}")

**Expected Output**:
>
```console
The model's output from 4 input values: 
[[-3.4       ]
 [-3.13333333]
 [-2.86666667]
 [-2.6       ]]
```

## 3 - Evaluate the Model
The Mean Squared Error (MSE) is a fundamental concept in statistics and machine learning, serving as a key metric for evaluating the performance of predictive models (linear model in this assignment). 
It quantifies the average of the squares of the errors between the predicted outcomes, $\hat{\mathbf{y}}$, and the ground true labels, $\mathbf{y}$.
$$\mathcal{L}(\hat{\mathbf{y}}, \mathbf{y}) = \frac{1}{M} \sum_{i=1}^M (^{(i)}\hat{ y} - ^{(i)}y)^2$$

### <font color='violet'>(10%) Exercise 3: Define Mean Square Error</font>
Please define a python function to compute MSE between the model predictions and true values from the dataset.

In [ ]:
def mse_loss(preds, labels):
    """
    Mean square error (MSE) function
        Args:
            preds: model predicted outcomes, 2d-array
            labels: true labels, 2d-array
        Returns:
            loss_value: averaged error between two groups, scalar
    """
    ### START CODE HERE ### (≈ 1 line of code)
    loss_value = None
    ### END CODE HERE ###
    return loss_value

# Sanity check
w = -0.8
b = 3.6
X_dummy = np.linspace(2, 4, 10).reshape(-1, 1)  # fake data for sanity check
y_dummy = np.linspace(10, 3, 10).reshape(-1, 1)
# Visualize the model
plt.plot(X_dummy, y_dummy, 'o')
plt.plot([0, 5], linear(np.array([0, 5]).reshape(-1, 1), w, b), 'r')
plt.legend(['dummy data', 'model'])
print(f"Model's MSE loss: {mse_loss(preds=linear(X_dummy, w, b), labels=y_dummy)}")

**Expected Output**:
>
```console
Model's MSE loss: 15.530000000000001
```

## 4 - Gradient Descent Optimization
To find out hints on improving the model (either increase or decrease the parameters), we need to calculate the derivatives of the loss function $\mathcal{L}$ with respect to $w$ and $b$ (__gradient__ of $\mathcal{L}$). 

$$\frac{\partial \mathcal{L}}{\partial w} = \frac{1}{M} \sum_{i=1}^M (^{(i)}\hat{ y} - ^{(i)}y) ^{(i)}x$$

$$\frac{\partial \mathcal{L}}{\partial b} = \frac{1}{M} \sum_{i=1}^M (^{(i)}\hat{ y} - ^{(i)}y)$$


### <font color='violet'>(20%) Exercise 4: Compute Loss Derivatives</font>

Please define a function to compute gradient (derivatives) of the loss: $\frac{\partial \mathcal{L}}{\partial w}$ and $\frac{\partial \mathcal{L}}{\partial b}$


In [ ]:
def grad(inputs, labels, preds):
    """
    Compute gradient of loss function
        Args:
            inputs: input features, 2d-array
            labels: groundtruths from dataset, 2d-array
            preds: model predicted outcomes, 2d-array
        Returns:
            dL_dw: dL/dw, scalar
            dL_db: dL/db, scalar
    """
    ### START CODE HERE ### (≈ 2 lines of code)
    dL_dw = None  # dL/dw
    dL_db = None  # dL/db
    ### END CODE HERE ###

    return dL_dw, dL_db

# Sanity check
dw, db = grad(inputs=X_dummy, labels=y_dummy, preds=linear(X_dummy, w, b))
print(f"dL/dw = {dw}, dL/db = {db}")

**Expected Output**:
>
```console
dL/dw = -14.8, dL/db = -5.300000000000001
```

### <font color='violet'>(40%) Exercise 5: Implement Gradient Descent Algorithm</font>
We could iterratively update weight and bias parameters to improve (or weaken) the model.
Please perform gradient descent algorithm and **improve** the model:


$\textbf{Initialize } w, b$

$\text{\textbf{Repeat} until converge } \{$

&nbsp; &nbsp; &nbsp; &nbsp; $\text{compute } \frac{\partial \mathcal{L}}{\partial w} \text{, and } \frac{\partial \mathcal{L}}{\partial b}$

&nbsp; &nbsp; &nbsp; &nbsp; $w := w - \alpha \frac{\partial \mathcal{L}}{\partial w}$

&nbsp; &nbsp; &nbsp; &nbsp; $b := b - \alpha \frac{\partial \mathcal{L}}{\partial b}$

$\}$

where $\alpha$ is the learning rate.

The goal is to bring the loss down **below 0.53**. 
> You may need to experiment this process a few times to find a good training profile of `num_iters` and `alpha`. 


In [ ]:
np.random.seed(3321)
# Initialize training data, linear model and storage
X_train = X_reshape
y_train = y_reshape
w = np.random.normal(loc=0, scale=1e-4)
b = np.random.normal(loc=0, scale=1e-4)
print(f"initial w = {w}, b = {b}")
losses = []  # storage for loss at each iteration

### START CODE HERE ### (≈ 7 lines of code)
num_iters = None
alpha = None  # learning rate
for i in range(num_iters):
    preds = None  # linear model predictions
    iter_loss = None  # evaluate model
    print(f"loss @ {i+1} iteration: {iter_loss}")
    losses.append(iter_loss)  # save loss 
    dw, db = None  # loss gradient w.r.t. weight, bias
    w = None  # update weight
    b = None  # update bias
### END CODE HERE ###
print(f"final w = {w}, b = {b}")

# To visualize changing of the loss
plt.xlabel("Iteration")
plt.ylabel("MSE loss")
plt.plot(losses, 'tan')
plt.legend(['MSE'])


## 5 - Evaluate and Test the Model
Let's first evaluate the model using the training dataset.


In [ ]:
print(f"On the training dataset, the model predicted price is different from the actual price: ${mse_loss(pred=linear(X_train, w, b), label=y_train) * 2 ** 0.5 * 1e4} in average")

# To visualize the model for how it fits to the training data
plt.plot(X_train, y_train, '.', markersize=1)
plt.plot([0, 5], linear(np.array([0, 5]).reshape(-1, 1), w, b), 'r')
plt.legend(['data', 'model'])


#### $\color{violet}{\textbf{(10\%) Exercise 6: Test Model with New Data}}$
Now, let's test the model with a new set of data.


In [ ]:
# Load test dataset
X_test = np.load("ages_test.npy").reshape(-1, 1) / 10  # rescale to per decade
y_test = np.load("prices_test.npy").reshape(-1, 1) / 1e4  # rescale to per $10,000
print(f"dimension of test features: {X_test.ndim}, shape of test features: {X_test.shape}")
print(f"dimension of test labels: {y_test.ndim}, shape of test labels: {y_test.shape}")

# Calculate mse loss of the model using the test data
### START CODE HERE ### (≈ 2 lines of code)
preds_test = None
loss_test = None
### END CODE HERE ###
print(f"On the test dataset, the model predicted price is different from the actual price: ${mse_loss(pred=linear(X_test, w, b), label=y_test) * 2 ** 0.5 * 1e4} in average")

# To visualize the model for how it fits to the test data
plt.xlabel("Age (decades)")
plt.ylabel("Price ($10,000)")
plt.plot(X_test, y_test, '.', markersize=1)
plt.plot([0, 5], linear(np.array([0, 5]), w, b), 'r')

> You may observe that the model did somehow catch the relationship between ages and prices of the cars. However, it does not make sense to predict negative price for the cars older than 3 decades. We'll investigate approaches to avoid this in the future.  

# Congratulations! You have finished this assignment!